In [1]:
# Basic lib imports
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

In [8]:
#AAL atlas paths
AAL_LEFT = '/opt/quarantine/CIVET/1.1.12/build/CIVET-1.1.12/models/AAL_atlas_left.txt'
AAL_RIGHT = '/opt/quarantine/CIVET/1.1.12/build/CIVET-1.1.12/models/AAL_atlas_right.txt'

AAL_lowres_threshold = 40962

In [9]:
# Return mean thickness per ROI for a given subject-vertex data
def apply_atlas(atlas_data,sub_data):
    assert len(atlas_data) == len(sub_data)
    label_set = np.unique(atlas_data)
    
    # mask subject data with atlas
    sub_roi_dict = {}
    for label in label_set:
        sub_roi_dist = sub_data[atlas_data==label]
        sub_roi_dict[label] = np.mean(sub_roi_dist)
        
    return sub_roi_dict

In [10]:
# Read atlas file
atlas_data = pd.read_csv(AAL_LEFT,header=None).values[:AAL_lowres_threshold]
label_set = np.unique(atlas_data)
print('# of unique labels: {}, max label: {}, min label: {}'.format(len(label_set),np.max(atlas_data),np.min(atlas_data)))

# Read subject file
subject_dir = '/data/chamal/projects/nikhil/data/ASD_ML/data/armin_data/civet_out/mallar_all/'
sub_file = '02493/thickness/nih_chp_02493_native_rms_rsl_tlink_30mm_left.txt'
sub_data = pd.read_csv(subject_dir + sub_file,header=None).values

sub_roi_dict = apply_atlas(atlas_data,sub_data)

# TODO
# convert to df so we can append multiple subjects

# of unique labels: 40, max label: 89, min label: 0
